In [42]:
import numpy as np
from scipy import signal
# from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from random import randint

In [43]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        raise NotImplementedError

    def backward(self, output_gradient, learning_rate):
        raise NotImplementedError

In [44]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient

In [45]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

In [46]:
class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)

        def tanh_prime(x):
            return 1 - np.tanh(x) ** 2

        super().__init__(tanh, tanh_prime)

In [47]:
class Loss:
    def loss(self, y_true, y_pred):
        raise NotImplementedError()

    def gradient(self, y_true, y_pred):
        raise NotImplementedError()

class MeanSquaredError(Loss):
    def loss(self, y_true, y_pred):
        return np.mean(np.power(y_true - y_pred, 2))

    def gradient(self, y_true, y_pred):
        return 2 * (y_pred - y_true) / y_true.size

In [48]:
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)

In [49]:
class BinaryCrossEntropy(Loss):
    def loss(self, y_true, y_pred):
        epsilon = 1e-7
        return -np.mean(np.multiply(y_true, np.log(y_pred + epsilon)) + np.multiply(1 - y_true, np.log(1 - y_pred + epsilon)))

    def gradient(self, y_true, y_pred):
        epsilon = 1e-7
        return np.divide(y_pred - y_true, np.multiply(y_pred + epsilon, 1 - y_pred + epsilon))

In [50]:
class Network(Layer):
    def __init__(self, layers):
        self.layers = layers
    
    def forward(self, input):
        output = input
        for layer in self.layers:
            output = layer.forward(output)
        return output
    
    def backward(self, output_error, learning_rate):
        for layer in reversed(self.layers):
            output_error = layer.backward(output_error, learning_rate)
        return output_error

class Model:
    def __init__(self, network, loss):
        self.network = Network(network)
        self.loss = loss
    
    def train(self, x_train, y_train, epochs, learning_rate, verbose=False):
        for epoch in range(epochs):
            for x, y in zip(x_train, y_train):
                output = self.network.forward(x)
                loss = self.loss.loss(y, output)
                loss_gradient = self.loss.gradient(y, output)
                self.network.backward(loss_gradient, learning_rate)
            if verbose:
                mean_loss = 0
                for x, y in zip(x_train, y_train):
                    output = self.network.forward(x)
                    loss = self.loss.loss(y, output)
                    mean_loss += loss
                mean_loss /= len(x_train)
                print(f"Epoch {epoch + 1}/{epochs} loss: {round(mean_loss, 4)}")
                test_model(self, x_train, y_train)
    
    def predict_one(self, x_test):
        return self.network.forward(x_test)

    def predict(self, x_test):
        y_pred = []
        for x in x_test:
            y_pred.append(self.network.forward(x))
        return np.array(y_pred).squeeze()

    def save(self, path):
        np.save(path, self.network)
    
    def load(self, path):
        self.network = np.load(path, allow_pickle=True)


In [51]:
# def preprocess_data(x, y, limit):
#     x = x.reshape(x.shape[0], 28 * 28, 1)
#     x = x.astype("float32") / 255
#     y = np.eye(10)[y]
#     y = y.reshape(y.shape[0], 10, 1)
#     return x[:limit], y[:limit]

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train, y_train = preprocess_data(x_train, y_train, 10000)
# x_test, y_test = preprocess_data(x_test, y_test, 1000)

# # neural network
# layers = [
#     Dense(28 * 28, 40),
#     Tanh(),
#     Dense(40, 10),
#     Sigmoid()
# ]

# model = Model(layers, BinaryCrossEntropy())
# model.train(x_train, y_train, 10, 0.1, verbose=True)

In [52]:
# correct = 0
# for x, y in zip(x_test, y_test):
#     output = model.predict_one(x)
#     # print("Prediction: %d, Ground Truth: %d" % (np.argmax(output), np.argmax(y)))
#     if np.argmax(output) == np.argmax(y):
#         correct += 1
# print("Accuracy: %.3f" % (correct / len(x_test)))

In [53]:
# choosen = randint(0, 1000)
# plt.imshow(x_test[choosen].reshape(28, 28), cmap="gray")
# prediction = model.predict_one(x_test[choosen])
# for i in range(10):
#     print(i, ":", round(prediction[i][0], 3))
# print("Prediction:", np.argmax(prediction))
# print("Ground Truth:", np.argmax(y_test[choosen]))


In [54]:
def test_model(model, x_test, y_test):
    correct = 0
    for x, y in zip(x_test, y_test):
        output = model.predict_one(x)
        # print("Output:", output)
        # print("Prediction: %d, Ground Truth: %d" % (np.argmax(output), np.argmax(y)))
        if np.argmax(output) == np.argmax(y):
            correct += 1
    print("Accuracy: %.3f" % (correct / len(x_test)))
# test_model(model, x_train, y_train)

In [55]:
# from Parsing import Parsing

# parser = Parsing()
# (x_train, y_train), (x_test, y_test) = parser.getData(0.8)

# def preprocess_data(x, y):
#     x = np.asarray(x)
#     x = x.reshape(x.shape[0], x.shape[1], 1)
#     y = np.asarray(y)
#     y = y.reshape(y.shape[0], y.shape[1], 1)
#     return x, y

# x_train, y_train = preprocess_data(x_train, y_train)
# x_test, y_test = preprocess_data(x_test, y_test)
# print("X train shape:", x_train.shape)
# print("Y train shape:", y_train.shape)
# print("X test shape:", x_test.shape)
# print("Y test shape:", y_test.shape)

In [56]:
# layers = [
#     Dense(389, 64),
#     Tanh(),
#     Dense(64, 4),
#     Sigmoid()
# ]

# model = Model(layers, BinaryCrossEntropy())
# model.train(x_train, y_train, 10, 0.1, verbose=True)

In [57]:
from parsingHenry import Ld

ld = Ld()
ld.loadinput("datasets/chess_positions.txt", "train")
print(ld.input.shape)
print(ld.target.shape)

(6000, 64)
(6000, 4)


In [58]:
def preprocess_data(x, y):
    x = x.reshape(x.shape[0], x.shape[1], 1)
    y = y.reshape(y.shape[0], y.shape[1], 1)
    return x, y

x_train, y_train = preprocess_data(ld.input, ld.target)

layers = [
    Dense(64, 200),
    Tanh(),
    Dense(200, 200),
    Tanh(),
    Dense(200, 4),
    Sigmoid()
]

model = Model(layers, BinaryCrossEntropy())
test_model(model, x_train, y_train)
model.train(x_train, y_train, 50, 0.0001, verbose=True)
test_model(model, x_train, y_train)

Accuracy: 0.233
Epoch 1/50 loss: 3.1695
Accuracy: 0.327
Epoch 2/50 loss: 2.6806
Accuracy: 0.380
Epoch 3/50 loss: 2.4024
Accuracy: 0.414
Epoch 4/50 loss: 2.1992
Accuracy: 0.442
Epoch 5/50 loss: 2.048
Accuracy: 0.465
Epoch 6/50 loss: 1.922
Accuracy: 0.485
Epoch 7/50 loss: 1.8131
Accuracy: 0.504
Epoch 8/50 loss: 1.7191
Accuracy: 0.518
Epoch 9/50 loss: 1.6345
Accuracy: 0.533
Epoch 10/50 loss: 1.5578
Accuracy: 0.548
Epoch 11/50 loss: 1.4888
Accuracy: 0.564
Epoch 12/50 loss: 1.4271
Accuracy: 0.574
Epoch 13/50 loss: 1.372
Accuracy: 0.585
Epoch 14/50 loss: 1.3207
Accuracy: 0.596
Epoch 15/50 loss: 1.272
Accuracy: 0.606
Epoch 16/50 loss: 1.2286
Accuracy: 0.617
Epoch 17/50 loss: 1.1885
Accuracy: 0.627
Epoch 18/50 loss: 1.151
Accuracy: 0.634
Epoch 19/50 loss: 1.1153
Accuracy: 0.642
Epoch 20/50 loss: 1.0821
Accuracy: 0.650
Epoch 21/50 loss: 1.05
Accuracy: 0.658
Epoch 22/50 loss: 1.0194
Accuracy: 0.663
Epoch 23/50 loss: 0.9911
Accuracy: 0.671
Epoch 24/50 loss: 0.965
Accuracy: 0.678
Epoch 25/50 loss:

In [67]:
total = 0
random_index = randint(0, len(x_train))
pred = model.predict_one(x_train[random_index])
real = y_train[random_index]

print("Random index:", random_index)
print(x_train[random_index].reshape(8, 8))
print("Prediction:", pred)
print("Real:", real)


Random index: 20
[[12  0  0  3  0  8  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  1]
 [ 0  0  0  0 10  7  0  0]
 [ 0  0  2  6  0  1  0  0]
 [ 0  0  0  0  0  0  0  0]]
Prediction: [[9.60293678e-01]
 [9.59951651e-01]
 [5.37284432e-04]
 [4.46223219e-01]]
Real: [[0]
 [0]
 [0]
 [1]]
